In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from IPython.display import clear_output

In [ ]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml') # make sure you have the HaarCascade

In [ ]:
cam = cv2.VideoCapture(0)

In [ ]:
# CATEGORIES = ["angry", "fear", "happy", "neutral", "sad", "surprise"] # comment if you want the YLYL one
CATEGORIES = ["not laughing", "not laughing", "laughing", "not laughing", "not laughing", "not laughing"] # comment if you want the normal one

model = tf.keras.models.load_model("C:/Users/Emmanuel/tf/projects/expresions cnn/models/expressionsV1.1.model") # you can load whatever model you want

IMG_SIZE = 50 # if you load a different model, make sure the dimensions here fit its input layer

lfc = 0 # laugh frame count. it will work of you laugh for more than 6 frames (for YLYL)

while True:
    ret, img = cam.read()
    height, width = img.shape[:2]
    font = cv2.FONT_HERSHEY_SIMPLEX
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    for (x,y,w,h) in faces:
        color = (0,225,0)
        if lfc > 0:
            color = (0, 0, 255)
            
            if lfc > 6:
                #os.system("shutdown /s /t 0") # do the funny thing, comment if you dont want the YLYL thing
                print("L")
        
        cv2.rectangle(img, (x,y), (x+w,y+h), color, 2)
        roi_gray = gray[y:y+h, x:x+w]

        roi_gray = cv2.resize(roi_gray, (IMG_SIZE, IMG_SIZE))
        
        sample = np.array(roi_gray).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
        sample = sample/255.0

        i = model.predict([sample])
        cv2.putText(img, str(CATEGORIES[np.argmax(i)]) ,(x,y+h+20), font, 1, color, 2, cv2.LINE_AA)

        if str(CATEGORIES[np.argmax(i)]) == "laughing": # works only for the YLYL one
            lfc += 1

        else: lfc = 0

    cv2.imshow('img', img)
    
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
        

cv2.destroyAllWindows()

In [ ]:
cam.release()